# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv("btc_historical_price.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df["date"] = pd.to_datetime(df["date"])

In [ ]:
df.head()

In [ ]:
df.set_index("date", inplace=True)

In [ ]:
df.head()

In [ ]:
df.plot()

In [ ]:
prices = df["price"]
prices = np.array(prices).reshape(-1, 1)

In [ ]:
scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(prices)

In [ ]:
train_size = int(len(prices_scaled) * 0.8)
test_size = len(prices_scaled) - train_size 

In [ ]:
print("train size:", train_size)
print("test size:", test_size)

In [ ]:
train = prices_scaled[0:train_size, :]
test = prices_scaled[train_size:len(prices_scaled), :]

In [ ]:
def dataset(data, steps=1):
    data_x, data_y = [], []
    for i in range(len(data) - steps - 1):
        a = data[i:(i + steps), 0]
        b = data[i + steps, 0]
        data_x.append(a)
        data_y.append(b)
    data_x = np.array(data_x)
    data_y = np.array(data_y)
    return data_x, data_y

In [ ]:
X_train, y_train = dataset(train)
X_test, y_test = dataset(test)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Model

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics =["accuracy"])

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3)

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=early_stopping)

In [ ]:
model.save("LSTM_BTC.h5")

In [ ]:
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = y_test.reshape(-1, 1)
y_test = scaler.inverse_transform(y_test)

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(y_test, label="Actual")
plt.plot(y_pred, label="Predicted")
plt.legend()
plt.show()

In [ ]:
r2_score(y_test, y_pred)